# Part 1: 오픈소스 Literature Agent 개관

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. 오픈소스 문헌 리뷰 에이전트의 종류와 특징 이해
2. 각 에이전트의 핵심 구성요소 파악
3. 학술 API (Semantic Scholar, arXiv, PubMed) 기본 사용법

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
# Colab: 자동 설치
# 로컬: pyproject.toml로 미리 설치 필요 (uv pip install -e . 또는 pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install requests python-dotenv semanticscholar arxiv google-generativeai -q
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")

In [ ]:
# Cell 3: API Key 로딩
import os
from pathlib import Path

# Colab 환경 체크
if IN_COLAB:
    # Colab userdata에서 키 불러오기
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        SEMANTIC_SCHOLAR_API_KEY = userdata.get('SEMANTIC_SCHOLAR_API_KEY', None)
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        print("⚠️ Colab Secrets에 GEMINI_API_KEY를 추가하세요")
        print("   좌측 🔑 아이콘 > Add new secret")
        GEMINI_API_KEY = None
        SEMANTIC_SCHOLAR_API_KEY = None
else:
    # 로컬 환경: dotenv 사용
    try:
        from dotenv import load_dotenv
        # 현재 디렉토리에서 .env 찾기
        load_dotenv()
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        SEMANTIC_SCHOLAR_API_KEY = os.getenv('SEMANTIC_SCHOLAR_API_KEY')
        
        if GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 GEMINI_API_KEY를 추가하세요")
            print("   예: GEMINI_API_KEY=your_key_here")
    except ImportError:
        print("⚠️ python-dotenv가 설치되지 않았습니다")
        print("   실행: pip install python-dotenv")
        GEMINI_API_KEY = None
        SEMANTIC_SCHOLAR_API_KEY = None

# API 사용 가능 여부 확인
print()
if GEMINI_API_KEY:
    print(f"✅ Gemini API Key: {GEMINI_API_KEY[:10]}...")
else:
    print("❌ Gemini API Key: 없음 (일부 기능 제한)")

if SEMANTIC_SCHOLAR_API_KEY:
    print(f"✅ Semantic Scholar API Key: {SEMANTIC_SCHOLAR_API_KEY[:10]}...")
else:
    print("\n⚠️  Semantic Scholar API Key가 설정되지 않았습니다")
    print("   → API 없이도 사용 가능하지만 rate limit 제한 (100 req/5분)")
    print("   → 발급 권장 (무료, 5000 req/5분): https://www.semanticscholar.org/product/api")

---
## 1. 파악하기: 오픈소스 Literature Agent 비교

코드가 공개된 에이전트만 다룹니다. 웹 서비스(Elicit, Consensus 등)는 제외.

### 주요 오픈소스 프로젝트 (참고용)

| 프로젝트 | Stars | 용도 | 특징 |
|----------|------:|------|------|
| [GPT-Researcher](https://github.com/assafelovic/gpt-researcher) | 24.9k | 웹검색 → 보고서 | 범용 리서치 |
| [PaperQA2](https://github.com/Future-House/paper-qa) | 8k | PDF RAG Q&A | 논문 기반 Q&A |
| [paperscraper](https://github.com/paperscraper-dev/paperscraper) | 300+ | 논문 메타데이터 수집 | 가벼움, API 래퍼 |
| [agentic-paper-review](https://github.com/forai-lab/agentic-paper-review) | 신규 | AI 논문 리뷰 | 구조화된 피드백 |

### 오늘 워크샵에서 다룰 내용

| 노트북 | 주제 | 핵심 도구 |
|--------|------|-----------|
| **1. Overview** | 학술 API 기본 사용법 | Semantic Scholar, arXiv, PubMed |
| **2. Crawlers** | Citation Crawler + Selector | paperscraper |
| **2.5. Manuscript** | 샘플 논문 준비 | Gemini API |
| **3. Review Agent** | AI 논문 리뷰 | agentic-paper-review |
| **4. Comparison** | 실제 피어리뷰 분석 | Transparent Peer Review 데이터 |

### 워크샵 전체 흐름

```
┌─────────────────────────────────────────────────────────────────┐
│  1. Overview        →   학술 API로 논문 검색 (SS, arXiv, PubMed) │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  2. Crawlers        →   Citation 네트워크 탐색 (paperscraper)    │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  2.5. Manuscript    →   AI 리뷰용 샘플 논문 준비 (Gemini)        │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  3. Review Agent    →   AI 논문 리뷰 받기 (agentic-paper-review) │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  4. Comparison      →   실제 피어리뷰와 비교 분석                 │
└─────────────────────────────────────────────────────────────────┘
```

### 핵심 구성요소

Literature Agent는 보통 다음 3가지 모듈로 구성됩니다:

1. **검색 모듈** - 논문 DB에서 관련 논문 찾기 (오늘 실습)
2. **처리 모듈** - 논문 파싱, 요약, 분석
3. **생성 모듈** - 결과물 생성 (리뷰, 보고서 등)

오늘 워크샵에서는 **검색 → 리뷰 생성** 파이프라인을 직접 구축해봅니다.

### 생각해보기

- 본인 연구에서 논문 검색/리뷰에 가장 시간이 많이 드는 부분은?
- AI 리뷰어에게 어떤 피드백을 받고 싶은가?

---
## 2. 써보기: 학술 API 기본 사용법

### API 비교

| API | Key 필요 | 비용 | 특징 |
|-----|----------|------|------|
| **Semantic Scholar** | 선택 | 무료 | 인용 네트워크, 임베딩 |
| **arXiv** | 불필요 | 무료 | 프리프린트, 전문 접근 |
| **PubMed** | 불필요 | 무료 | 의학/심리학 특화 |

### 학술 API 함수 정의

세 가지 API 검색 함수를 먼저 정의합니다. (한 번만 실행)

In [ ]:
# Cell: 학술 API 함수 정의 (Semantic Scholar, arXiv, PubMed)
import requests
import arxiv

# ─────────────────────────────────────────
# 1. Semantic Scholar API
# ─────────────────────────────────────────
def search_semantic_scholar(query, limit=5):
    """Semantic Scholar에서 논문 검색"""
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": limit,
        "fields": "paperId,title,authors,year,abstract,citationCount,externalIds"
    }
    
    headers = {}
    if SEMANTIC_SCHOLAR_API_KEY:
        headers['x-api-key'] = SEMANTIC_SCHOLAR_API_KEY
    
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 429:
        print("⚠️ Semantic Scholar API rate limit 초과")
        return {"data": []}
    elif response.status_code != 200:
        print(f"⚠️ API 오류: {response.status_code}")
        return {"data": []}
    
    return response.json()

# ─────────────────────────────────────────
# 2. arXiv API
# ─────────────────────────────────────────
def search_arxiv(query, max_results=5):
    """arXiv에서 논문 검색"""
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    return list(client.results(search))

# ─────────────────────────────────────────
# 3. PubMed API (심리학/의학 특화)
# ─────────────────────────────────────────
def search_pubmed(query, max_results=5):
    """PubMed에서 논문 검색"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    
    # 1. 검색
    search_url = f"{base_url}esearch.fcgi"
    search_params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "json"
    }
    search_result = requests.get(search_url, params=search_params).json()
    ids = search_result.get('esearchresult', {}).get('idlist', [])
    
    if not ids:
        return []
    
    # 2. 상세 정보
    fetch_url = f"{base_url}esummary.fcgi"
    fetch_params = {
        "db": "pubmed",
        "id": ",".join(ids),
        "retmode": "json"
    }
    fetch_result = requests.get(fetch_url, params=fetch_params).json()
    
    papers = []
    for pid in ids:
        info = fetch_result.get('result', {}).get(pid, {})
        # DOI 추출
        doi = None
        for article_id in info.get('articleids', []):
            if article_id.get('idtype') == 'doi':
                doi = article_id.get('value')
                break
        papers.append({
            'pmid': pid,
            'title': info.get('title', ''),
            'authors': [a.get('name', '') for a in info.get('authors', [])],
            'year': info.get('pubdate', '')[:4],
            'journal': info.get('source', ''),
            'doi': doi
        })
    return papers

# ─────────────────────────────────────────
print("✅ API 함수 정의 완료!")
print("   - search_semantic_scholar(query)")
print("   - search_arxiv(query)")
print("   - search_pubmed(query)")

In [ ]:
# DIY Cell: 검색 설정 (이후 모든 셀에 적용)
# ============================================================
# ✏️ 아래 값들을 수정하세요!
# ============================================================

MY_QUERY = "your research topic here"  # 검색 주제
MY_YEAR_FROM = 2020                     # 최소 연도
MY_MIN_CITATIONS = 50                   # 최소 인용수

# ============================================================

# 입력 검증
if MY_QUERY == "your research topic here":
    print("=" * 60)
    print("⚠️  MY_QUERY를 수정하세요!")
    print("=" * 60)
    print()
    print("예시:")
    print('   MY_QUERY = "depression social media adolescent"')
    print('   MY_QUERY = "large language model psychology"')
    print('   MY_QUERY = "oppositional defiant disorder treatment"')
    print()
else:
    print("✅ 검색 설정 완료!")
    print(f"   📝 주제: {MY_QUERY}")
    print(f"   📅 연도: {MY_YEAR_FROM}년 이후")
    print(f"   📊 최소 인용수: {MY_MIN_CITATIONS}회")
    print()
    print("→ 다음 셀을 실행하여 검색 결과를 확인하세요.")

---
## 3. 바꿔보기: 검색 함수 커스텀

In [ ]:
# Cell: 고급 검색 + 필터링 (설정값 사용)
# ============================================================
# 필터 옵션 (True/False로 켜고 끄기)
# ============================================================

USE_YEAR_FILTER = True        # 연도 필터
USE_CITATION_FILTER = False   # 인용수 필터 (심리학 논문은 낮게 설정 권장)
# USE_VENUE_FILTER = True     # 저널 필터 (아래 TOP_VENUES 설정 필요)

# 저널 필터용 (USE_VENUE_FILTER = True 시 사용)
# TOP_VENUES = ["Journal of Abnormal Child Psychology", "JAMA Psychiatry", "JAACAP"]

# ============================================================

def search_advanced(query, year_from=2020, min_citations=10):
    """필터링 옵션이 추가된 Semantic Scholar 검색"""
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": 20,
        "fields": "paperId,title,authors,year,abstract,citationCount,venue,externalIds",
    }
    
    if USE_YEAR_FILTER:
        params["year"] = f"{year_from}-"
    
    headers = {}
    if SEMANTIC_SCHOLAR_API_KEY:
        headers['x-api-key'] = SEMANTIC_SCHOLAR_API_KEY
    
    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        print(f"⚠️ API 오류 ({response.status_code})")
        return []
    
    papers = response.json().get('data', [])
    
    # 필터 적용
    if USE_CITATION_FILTER:
        papers = [p for p in papers if p.get('citationCount', 0) >= min_citations]
    
    # if USE_VENUE_FILTER and 'TOP_VENUES' in dir():
    #     papers = [p for p in papers 
    #               if any(v.lower() in str(p.get('venue', '')).lower() for v in TOP_VENUES)]
    
    return papers

# ─────────────────────────────────────────
# 실행
# ─────────────────────────────────────────
if 'MY_QUERY' not in dir() or MY_QUERY == "your research topic here":
    print("❌ 위의 '검색 설정' 셀을 먼저 실행하세요!")
else:
    print(f"📝 검색 주제: {MY_QUERY}")
    print(f"📅 연도: {MY_YEAR_FROM}년 이후" if USE_YEAR_FILTER else "📅 연도: 전체")
    print(f"📊 최소 인용: {MY_MIN_CITATIONS}회" if USE_CITATION_FILTER else "📊 인용수 필터: 꺼짐")
    print("=" * 60)
    
    # Semantic Scholar (필터 적용)
    ss_filtered = search_advanced(MY_QUERY, MY_YEAR_FROM, MY_MIN_CITATIONS)
    print(f"\n🔬 Semantic Scholar: {len(ss_filtered)}개")
    for p in ss_filtered[:5]:
        print(f"   📄 {p['title'][:60]}...")
        ext_ids = p.get('externalIds') or {}
        doi = ext_ids.get('DOI')
        print(f"      {p.get('year', 'N/A')} | 인용: {p['citationCount']}")
        if doi:
            print(f"      🔗 https://doi.org/{doi}")
    
    # arXiv
    ax = search_arxiv(MY_QUERY, max_results=5)
    print(f"\n📚 arXiv: {len(ax)}개")
    for paper in ax[:5]:
        print(f"   📄 {paper.title[:60]}...")
        print(f"      🔗 {paper.entry_id}")
    
    # PubMed
    pm = search_pubmed(MY_QUERY, max_results=5)
    print(f"\n🏥 PubMed: {len(pm)}개")
    for p in pm[:5]:
        print(f"   📄 {p['title'][:60]}...")
        if p.get('doi'):
            print(f"      🔗 https://doi.org/{p['doi']}")
        else:
            print(f"      🔗 https://pubmed.ncbi.nlm.nih.gov/{p['pmid']}/")

---
## 4. 토론

**커스텀하고 싶은 기능은?**
- 한글 논문 검색?
- 특정 저널 필터?
- 메타분석용 효과크기 추출?